In [ ]:
import pandas as pd
import numpy as np
import folium
import pylandstats as pls
import geopandas as gpd


cor1='#090571'
cor2='#ffcc00'

In [68]:
# CIDADES
csv_path_lda = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_londrina/inference_results_with_positions_20250609_083252.csv'
csv_path_ibip ='/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_ibipora/inference_results_with_positions_20250609_164529.csv'
csv_path_camb = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambe/inference_results_with_positions_20250609_161440.csv'
csv_path_apuc = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_apucarana/inference_results_with_positions_20251118_114239.csv'
csv_path_arap = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_arapongas/inference_results_with_positions_20251118_121155.csv'
csv_path_cambir= '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambira/inference_results_with_positions_20251118_122253.csv'
csv_path_jand = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_jandaia/inference_results_with_positions_20251118_122910.csv'
csv_path_mand = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_mandaguari/inference_results_with_positions_20251118_125143.csv'
csv_path_mari = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_marialva/inference_results_with_positions_20251118_124329.csv'
csv_path_rol = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_rolandia/inference_results_with_positions_20251118_130921.csv'
csv_path_sar = '/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_sarandi/inference_results_with_positions_20251118_131430.csv'
csv_path_maring ='/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_maringa/inference_results_with_positions_20251118_133121.csv'

csv_path=csv_path_maring
city="maringa"
df = pd.read_csv(csv_path, sep=';')
print(f"Processando {len(df):,} patches...")

Processando 11,481 patches...


In [ ]:
#######################################################
# MAPA DA AGREGAÇÃO DAS 10 CLASSES EM DUAS 
#######################################################


# DEFINIÇÕES DE AGREGAÇÃO
CLASSE_NOMES = {
    'hduf': 'Alta Densidade Urbana',
    'lduf': 'Baixa Densidade Urbana',
    'mduf': 'Média Densidade Urbana',
    'industrial': 'Industrial',
    'crop': 'Cultivo',
    'bush': 'Arbustivo',
    'tree': 'Floresta',
    'bare': 'Solo Exposto',
    'water': 'Água',
    'grass': 'Gramíneas'
}

CLASSE_CONSTRUIDO = ['hduf', 'lduf', 'mduf', 'industrial']
CLASSE_NAO_CONSTRUIDO = ['crop', 'bush', 'tree', 'bare', 'water', 'grass']

MAPEAMENTO_NUMERICO = {
    'hduf': 1, 'lduf': 1, 'mduf': 1, 'industrial': 1,
    'crop': 2, 'bush': 2, 'tree': 2, 'bare': 2, 'water': 2, 'grass': 2
}

CLASSE_INDUSTRIAL = ['industrial']
CLASSE_OUTRAS = ['hduf', 'lduf', 'mduf', 'crop', 'bush', 'tree', 'bare', 'water', 'grass']
MAPEAMENTO_NUMERICO2 = {
    'hduf': 2, 'lduf': 2, 'mduf': 2, 'industrial': 1,
    'crop': 2, 'bush': 2, 'tree': 2, 'bare': 2, 'water': 2, 'grass': 2
}


def agregar_classes(df, coluna_classe='predicted_class'):
    """Agrega classes em Construído (1) e Não Construído (2)."""
    df = df.copy()
    # df['classe_agregada'] = df[coluna_classe].map(MAPEAMENTO_NUMERICO)
    df['classe_agregada'] = df[coluna_classe].map(MAPEAMENTO_NUMERICO2)
    
    
    nao_mapeadas = df[df['classe_agregada'].isna()]
    if len(nao_mapeadas) > 0:
        classes_problema = nao_mapeadas[coluna_classe].unique()
        print(f"⚠️ {len(nao_mapeadas)} patches não mapeados: {classes_problema}")
        df = df.dropna(subset=['classe_agregada'])
    
    df['classe_agregada'] = df['classe_agregada'].astype(int)
    
    print(f"✓ Construído: {len(df[df['classe_agregada'] == 1])} patches")
    print(f"✓ Não Construído: {len(df[df['classe_agregada'] == 2])} patches")
    
    return df

def calcular_cantos_quadrado(lat_center, lon_center, lado_metros=109.45):
    """
    Calcula os 4 cantos de um quadrado a partir do centro.
    
    Parameters
    ----------
    lat_center : float
        Latitude do centro
    lon_center : float
        Longitude do centro
    lado_metros : float
        Lado do quadrado em metros
    
    Returns
    -------
    list
        Lista com 4 coordenadas [lat, lon] dos cantos
    """
    # Metade do lado
    half_side = lado_metros / 2.0
    
    # Converter para graus
    # 1 grau latitude ≈ 111,000 metros
    lat_offset = half_side / 111000.0
    
    # 1 grau longitude depende da latitude
    lon_offset = half_side / (111000.0 * np.cos(np.radians(lat_center)))
    
    # Calcular os 4 cantos (sentido horário a partir do canto superior esquerdo)
    cantos = [
        [lat_center + lat_offset, lon_center - lon_offset],  # Superior esquerdo
        [lat_center + lat_offset, lon_center + lon_offset],  # Superior direito
        [lat_center - lat_offset, lon_center + lon_offset],  # Inferior direito
        [lat_center - lat_offset, lon_center - lon_offset],  # Inferior esquerdo
    ]
    
    return cantos


# 1. Carregar CSV
# carregar a codade no começo

# 2. Parsear coordenadas
def parse_coords(s):
    lat, lon = map(float, str(s).split(','))
    return lat, lon

coords = df['center'].apply(parse_coords)
df['latitude'] = coords.apply(lambda x: x[0])
df['longitude'] = coords.apply(lambda x: x[1])

# 3. Agregar classes
df = agregar_classes(df, coluna_classe='predicted_class')

# 4. Criar mapa
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)




# 5. Adicionar quadrados
print("\nAdicionando quadrados ao mapa...")

for idx, row in df.iterrows():
    if row['classe_agregada'] == 1:  # Construído/industrial
        color = cor1 #'#d62728'  # Vermelho
        # categoria = 'Construído'
        categoria = 'Industrial'
        
    else:  # Não Construído/outra
        color = cor2  #'#2ca02c'  # Verde
        categoria = 'Other'
    
    
    # Calcular cantos do quadrado
    cantos = calcular_cantos_quadrado(row['latitude'], row['longitude'], lado_metros=109.45)
    
    # Criar polígono (quadrado)
    folium.Polygon(
        locations=cantos,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6,
        weight=1,
        popup=f"<b>{categoria}</b><br>{CLASSE_NOMES[row['predicted_class']]}"
    ).add_to(m)
    
    if idx % 1000 == 0:
        print(f"  {idx:,}/{len(df):,} patches")

# 6. Legenda
legend_html = """
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 200px; height: 180px; 
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; border-radius: 10px;
            padding: 10px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <p style="margin: 10px; font-weight: bold; font-size: 16px;">
        Urban land cover class segmentation
    </p>
    <p style="margin: 5px;">
        <span style="background-color: #d62728; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>Industrial</b>
    </p>
    <p style="margin: 5px;">
        <span style="background-color: #2ca02c; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>Other</b>
    </p>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

print("\n✓ Mapa com quadrados pronto!")

output_file = f'saida_html_patches/mapa_agregado_industrial_{city}.html'
m.save(output_file)


✓ Construído: 1951 patches
✓ Não Construído: 9530 patches

Adicionando quadrados ao mapa...
  0/11,481 patches
  1,000/11,481 patches
  2,000/11,481 patches
  3,000/11,481 patches
  4,000/11,481 patches
  5,000/11,481 patches
  6,000/11,481 patches
  7,000/11,481 patches
  8,000/11,481 patches
  9,000/11,481 patches
  10,000/11,481 patches
  11,000/11,481 patches

✓ Mapa com quadrados pronto!


In [70]:
#######################################################
# MAPA DA AGREGAÇÃO DAS 10 CLASSES EM DUAS 
#######################################################



# DEFINIÇÕES
MAPEAMENTO_NUMERICO = {
    'hduf': 1, 'lduf': 1, 'mduf': 1, 'industrial': 1,
    'crop': 2, 'bush': 2, 'tree': 2, 'bare': 2, 'water': 2, 'grass': 2
}

MAPEAMENTO_NUMERICO2 = {
    'hduf': 2, 'lduf': 2, 'mduf': 2, 'industrial': 1,
    'crop': 2, 'bush': 2, 'tree': 2, 'bare': 2, 'water': 2, 'grass': 2
}


# 1. Carregar e preparar
#carregar a cidade nocomeço

# 2. Coordenadas
coords = df['center'].apply(lambda s: tuple(map(float, str(s).split(','))))
df['latitude'] = coords.apply(lambda x: x[0])
df['longitude'] = coords.apply(lambda x: x[1])

# 3. Agregar
# df['classe_agregada'] = df['predicted_class'].map(MAPEAMENTO_NUMERICO)
df['classe_agregada'] = df['predicted_class'].map(MAPEAMENTO_NUMERICO2)

df = df.dropna(subset=['classe_agregada'])
df['classe_agregada'] = df['classe_agregada'].astype(int)

print(f"✓ Construído: {len(df[df['classe_agregada'] == 1]):,} patches")

# 4. Criar raster
min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
min_lon, max_lon = df['longitude'].min(), df['longitude'].max()

tamanho_patch = 109.45  # metros
metros_por_grau = 111000 * np.cos(np.radians(min_lat))
patch_deg = tamanho_patch / metros_por_grau

n_cols = int(np.ceil((max_lon - min_lon) / patch_deg)) + 1
n_rows = int(np.ceil((max_lat - min_lat) / patch_deg)) + 1
raster = np.zeros((n_rows, n_cols), dtype=np.int32)

for _, row in df.iterrows():
    col = int((row['longitude'] - min_lon) / patch_deg)
    r = int((max_lat - row['latitude']) / patch_deg)
    if 0 <= r < n_rows and 0 <= col < n_cols:
        raster[r, col] = row['classe_agregada']

print(f"✓ Raster criado: {raster.shape}")

# 5. PyLandStats com conectividade de 4 vizinhos
landscape = raster.copy().astype(float)
landscape[landscape == 0] = np.nan
ls = pls.Landscape(landscape, res=(tamanho_patch, tamanho_patch), neighbourhood_rule=4)

print("✓ Usando conectividade de 4 vizinhos (apenas lados)")

# 6. Ver estrutura do DataFrame primeiro
class_metrics = ls.compute_class_metrics_df()
print("\nColunas disponíveis:")
print(class_metrics.columns.tolist())
print("\nDataFrame:")
print(class_metrics)

# Filtrar classe construída (1)
# Tentar diferentes possíveis nomes de coluna
if 'class_val' in class_metrics.columns:
    construido = class_metrics[class_metrics['class_val'] == 1].iloc[0]
elif 'class' in class_metrics.columns:
    construido = class_metrics[class_metrics['class'] == 1].iloc[0]
else:
    # Usar índice direto se tiver apenas 2 linhas
    construido = class_metrics.iloc[0]  # Assumir que primeira linha é classe 1

print("\nMétricas da classe Construído:")
print(construido)

# Tentar diferentes nomes para perímetro
if 'total_edge' in construido.index:
    perimetro_total = construido['total_edge']
elif 'te' in construido.index:
    perimetro_total = construido['te']
elif 'total_edge_length' in construido.index:
    perimetro_total = construido['total_edge_length']
else:
    print("\n⚠️ Coluna de perímetro não encontrada. Colunas disponíveis:")
    print(construido.index.tolist())
    perimetro_total = None

if perimetro_total is not None:
    print("\n" + "="*60)
    print("PERÍMETRO DA ÁREA CONSTRUÍDA")
    print("="*60)
    print(f"\nConectividade: Apenas pelos LADOS (4-connectivity)")
    print(f"\nPerímetro total: {perimetro_total:,.2f} metros")
    print(f"                 {perimetro_total/1000:,.2f} km")
    print("="*60)

output_file = f'saida_html_patches/mapa_agregado_indutrial_{city}B.html'
m.save(output_file)

✓ Construído: 1,951 patches
✓ Raster criado: (119, 163)
✓ Usando conectividade de 4 vizinhos (apenas lados)

Colunas disponíveis:
['total_area', 'proportion_of_landscape', 'number_of_patches', 'patch_density', 'largest_patch_index', 'total_edge', 'edge_density', 'total_core_area', 'core_area_proportion_of_landscape', 'number_of_disjunct_core_areas', 'landscape_shape_index', 'effective_mesh_size', 'area_mn', 'area_am', 'area_md', 'area_ra', 'area_sd', 'area_cv', 'perimeter_mn', 'perimeter_am', 'perimeter_md', 'perimeter_ra', 'perimeter_sd', 'perimeter_cv', 'perimeter_area_ratio_mn', 'perimeter_area_ratio_am', 'perimeter_area_ratio_md', 'perimeter_area_ratio_ra', 'perimeter_area_ratio_sd', 'perimeter_area_ratio_cv', 'shape_index_mn', 'shape_index_am', 'shape_index_md', 'shape_index_ra', 'shape_index_sd', 'shape_index_cv', 'fractal_dimension_mn', 'fractal_dimension_am', 'fractal_dimension_md', 'fractal_dimension_ra', 'fractal_dimension_sd', 'fractal_dimension_cv', 'core_area_mn', 'core_ar

In [ ]:
#######################################################
# MAPA DA INDIVIDUALIZAÇÃO DOS PATCHES CONSTRUÍDOS
# (USANDO COORDENADAS MÉTRICAS EPSG:29192)
#######################################################

# 0. DEFINIÇÕES
MAPEAMENTO_NUMERICO = {
    'hduf': 1, 'lduf': 1, 'mduf': 1, 'industrial': 1,
    'crop': 2, 'bush': 2, 'tree': 2, 'bare': 2, 'water': 2, 'grass': 2
}
MAPEAMENTO_NUMERICO2 = {
    'hduf': 2, 'lduf': 2, 'mduf': 2, 'industrial': 1,
    'crop': 2, 'bush': 2, 'tree': 2, 'bare': 2, 'water': 2, 'grass': 2
}


CLASSE_NOMES = {
    'hduf': 'Alta Densidade Urbana',
    'lduf': 'Baixa Densidade Urbana',
    'mduf': 'Média Densidade Urbana',
    'industrial': 'Industrial',
    'crop': 'Cultivo',
    'bush': 'Arbustivo',
    'tree': 'Floresta',
    'bare': 'Solo Exposto',
    'water': 'Água',
    'grass': 'Gramíneas'
}

CORES_FACES = {
    0: '#d62728',  # Vermelho - interno
    1: "#256825",  # Verde - 1 face exposta
    2: '#1f77b4',  # Azul - 2 faces expostas
    3: "#6e6e5f",  # Amarelo - 3 faces expostas
    4: '#9467bd'   # Roxo - isolado (4 faces)
}

NOMES_FACES = {
    0: 'Interno (0 faces)',
    1: '1 face exposta',
    2: '2 faces expostas',
    3: '3 faces expostas',
    4: 'Isolado (4 faces)'
}

tamanho_patch = 109.45  # metros (lado do patch)
cell_size = tamanho_patch

def calcular_cantos_quadrado(lat_center, lon_center, lado_metros=109.45):
    """Calcula os 4 cantos de um quadrado em coordenadas geográficas."""
    half_side = lado_metros / 2.0
    lat_offset = half_side / 111000.0
    lon_offset = half_side / (111000.0 * np.cos(np.radians(lat_center)))
    
    return [
        [lat_center + lat_offset, lon_center - lon_offset],  # sup esq
        [lat_center + lat_offset, lon_center + lon_offset],  # sup dir
        [lat_center - lat_offset, lon_center + lon_offset],  # inf dir
        [lat_center - lat_offset, lon_center - lon_offset],  # inf esq
    ]

def contar_faces_expostas(raster, row, col):
    """
    Conta faces expostas para um patch antrópico (valor 1) em raster.
    Vizinhança 4 (cima, baixo, esquerda, direita).
    """
    n_rows, n_cols = raster.shape
    faces_expostas = 0
    vizinhos = [
        (row - 1, col),  # Cima
        (row + 1, col),  # Baixo
        (row, col - 1),  # Esquerda
        (row, col + 1)   # Direita
    ]
    
    for vr, vc in vizinhos:
        # Fora do raster => exposta
        if vr < 0 or vr >= n_rows or vc < 0 or vc >= n_cols:
            faces_expostas += 1
        elif raster[vr, vc] != 1:  # não é antrópico (0 vazio ou 2 natural)
            faces_expostas += 1
    
    return faces_expostas

def parse_coords(s):
    lat, lon = map(float, str(s).split(','))
    return lat, lon

#######################################################
# 1. PREPARO DAS COORDENADAS
#######################################################

# df deve estar carregado e conter a coluna 'center' = "lat,lon"
coords = df['center'].apply(parse_coords)
df['latitude'] = coords.apply(lambda x: x[0])
df['longitude'] = coords.apply(lambda x: x[1])

# GeoDataFrame em EPSG:4326
gdf = gpd.GeoDataFrame(
    df.copy(),
    geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
    crs="EPSG:4326"
)

# Reprojetar para EPSG:29192 (coordenadas métricas)
gdf_utm = gdf.to_crs("EPSG:29192")
gdf_utm['x_utm'] = gdf_utm.geometry.x
gdf_utm['y_utm'] = gdf_utm.geometry.y

#######################################################
# 2. AGREGAÇÃO EM 2 CLASSES (ANTRÓPICO / NATURAL)
#######################################################

# gdf_utm['classe_agregada'] = gdf_utm['predicted_class'].map(MAPEAMENTO_NUMERICO)
gdf_utm['classe_agregada'] = gdf_utm['predicted_class'].map(MAPEAMENTO_NUMERICO2)

gdf_utm = gdf_utm.dropna(subset=['classe_agregada'])
gdf_utm['classe_agregada'] = gdf_utm['classe_agregada'].astype(int)

total_antropico = len(gdf_utm[gdf_utm['classe_agregada'] == 1])
total_natural   = len(gdf_utm[gdf_utm['classe_agregada'] == 2])

print(f"✓ Antrópico: {total_antropico} patches")
print(f"✓ Natural  : {total_natural} patches")

#######################################################
# 3. INDEXAÇÃO EM GRADE MÉTRICA (GARANTIR 1 CÉLULA/PATCH)
#######################################################

x0 = gdf_utm['x_utm'].min()
y_max = gdf_utm['y_utm'].max()  # origem no topo

gdf_utm['col_index'] = ((gdf_utm['x_utm'] - x0) / cell_size).round().astype(int)
gdf_utm['row_index'] = ((y_max - gdf_utm['y_utm']) / cell_size).round().astype(int)

# Verificar colisões apenas em patches antrópicos
antropicos = gdf_utm[gdf_utm['classe_agregada'] == 1]
dup_cells = (antropicos
             .groupby(['row_index', 'col_index'])
             .size()
             .reset_index(name='n'))

tem_colisao = (dup_cells['n'] > 1).any()
if tem_colisao:
    print("\n⚠️ Existem colisões de mais de um patch antrópico na mesma célula:")
    print(dup_cells[dup_cells['n'] > 1].head())
else:
    print("\n✓ Não há colisões: 1 patch antrópico por célula de grade.")

#######################################################
# 4. CONSTRUÇÃO DO RASTER (USANDO row_index / col_index)
#######################################################

n_rows = gdf_utm['row_index'].max() + 1
n_cols = gdf_utm['col_index'].max() + 1

raster = np.zeros((n_rows, n_cols), dtype=np.int32)
coord_to_index = {}

for _, row in gdf_utm.iterrows():
    r = int(row['row_index'])
    c = int(row['col_index'])
    if 0 <= r < n_rows and 0 <= c < n_cols:
        raster[r, c] = row['classe_agregada']
        # guardamos também lat/lon e classe original para o mapa
        coord_to_index[(r, c)] = (
            float(row['latitude']),
            float(row['longitude']),
            row['predicted_class']
        )

print(f"\n✓ Raster criado: {raster.shape}")

#######################################################
# 5. CÁLCULO DAS FACES EXPOSTAS (APENAS ANTRÓPICOS)
#######################################################

print("\nCalculando faces expostas...")
faces_expostas_map = {}
for (r, c), (lat, lon, classe_orig) in coord_to_index.items():
    if raster[r, c] == 1:  # apenas antrópicos
        n_faces = contar_faces_expostas(raster, r, c)
        faces_expostas_map[(r, c)] = n_faces

# Estatísticas por número de faces
stats = {k: 0 for k in range(5)}
for n_faces in faces_expostas_map.values():
    stats[n_faces] += 1

total_antropico_faces = sum(stats.values())

print("\nDistribuição de patches antrópicos por faces expostas:")
for k in range(5):
    perc = stats[k] / total_antropico_faces * 100 if total_antropico_faces > 0 else 0.0
    print(f"  {k} faces: {stats[k]:3d} patches ({perc:4.1f}%)")

print(f"\nTotal antrópico no df          : {total_antropico}")
print(f"Total antrópico com faces_expostas: {total_antropico_faces}")

#######################################################
# 6. MAPA FOLIUM PARA INSPEÇÃO VISUAL
#######################################################

center_lat = gdf_utm['latitude'].mean()
center_lon = gdf_utm['longitude'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)

print("\nAdicionando patches ao mapa...")

# Natural (classe 2) em cinza claro
for (r, c), (lat, lon, classe_orig) in coord_to_index.items():
    if raster[r, c] == 2:
        cantos = calcular_cantos_quadrado(lat, lon, tamanho_patch)
        folium.Polygon(
            locations=cantos,
            color='#cccccc',
            fill=True,
            fillColor='#cccccc',
            fillOpacity=0.3,
            weight=0.5,
            popup=f"Other<br>{CLASSE_NOMES[classe_orig]}"
        ).add_to(m)

# Antrópico (classe 1) colorido por faces expostas
contador = 0
for (r, c), (lat, lon, classe_orig) in coord_to_index.items():
    if raster[r, c] == 1:
        n_faces = faces_expostas_map[(r, c)]
        cor = CORES_FACES[n_faces]
        cantos = calcular_cantos_quadrado(lat, lon, tamanho_patch)
        folium.Polygon(
            locations=cantos,
            color=cor,
            fill=True,
            fillColor=cor,
            fillOpacity=0.7,
            weight=1,
            popup=f"<b>Industrial - {NOMES_FACES[n_faces]}</b><br>{CLASSE_NOMES[classe_orig]}"
        ).add_to(m)
        contador += 1
        if contador % 1000 == 0:
            print(f"  {contador:,}/{total_antropico_faces:,} patches antrópicos")

#######################################################
# 7. LEGENDA E SALVAMENTO
#######################################################

legend_html = f"""
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 260px; 
            background-color: white; z-index:9999; font-size:13px;
            border:2px solid grey; border-radius: 10px;
            padding: 15px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <p style="margin: 0 0 10px 0; font-weight: bold; font-size: 15px;">
        Exposed edges (Industrial)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: {CORES_FACES[0]}; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>0</b> Intern patch ({stats[0]:,})
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: {CORES_FACES[1]}; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>1</b> Exposed edge ({stats[1]:,})
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: {CORES_FACES[2]}; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>2</b> Exposed edges ({stats[2]:,})
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: {CORES_FACES[3]}; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>3</b> Exposed edges ({stats[3]:,})
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: {CORES_FACES[4]}; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        <b>4</b> Isolated patches ({stats[4]:,})
    </p>
    <hr style="margin: 10px 0;">
    <p style="margin: 5px 0; font-size: 11px; color: #666;">
        <span style="background-color: #cccccc; 
                     width: 20px; height: 20px; 
                     display: inline-block; 
                     border: 1px solid black;"></span>
        Other
    </p>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

print("\n✓ Mapa pronto!")
print("  🔴 Vermelho - Interno (0 faces)")
print("  🟢 Verde    - 1 face exposta")
print("  🔵 Azul     - 2 faces expostas")
print("  🟡 Amarelo  - 3 faces expostas")
print("  🟣 Roxo     - Isolado (4 faces)")

output_file = f'saida_html_patches/mapa_individualizacao_patches_industrial_{city}.html'
m.save(output_file)
print(f"\nHTML salvo em: {output_file}")


✓ Antrópico: 1951 patches
✓ Natural  : 9530 patches

⚠️ Existem colisões de mais de um patch antrópico na mesma célula:
     row_index  col_index  n
795         60        127  2

✓ Raster criado: (129, 163)

Calculando faces expostas...

Distribuição de patches antrópicos por faces expostas:
  0 faces: 341 patches (17.5%)
  1 faces: 493 patches (25.3%)
  2 faces: 503 patches (25.8%)
  3 faces: 375 patches (19.3%)
  4 faces: 236 patches (12.1%)

Total antrópico no df          : 1951
Total antrópico com faces_expostas: 1948

Adicionando patches ao mapa...
  1,000/1,948 patches antrópicos

✓ Mapa pronto!
  🔴 Vermelho - Interno (0 faces)
  🟢 Verde    - 1 face exposta
  🔵 Azul     - 2 faces expostas
  🟡 Amarelo  - 3 faces expostas
  🟣 Roxo     - Isolado (4 faces)

HTML salvo em: saida_html_patches/mapa_individualizacao_patches_industrial_maringa.html


In [72]:
area=(stats[0]+stats[1]+stats[2]+stats[3]+stats[4])*0.10945*0.10945
area
per=(stats[0]*0+stats[1]*1+stats[2]*2+stats[3]*3+stats[4]*4)*0.10945
per
per/area

# Circulo ideal
A = area  
r_circ = np.sqrt(A / np.pi)          # raio do círculo equivalente
P_circ = 2 * np.pi * r_circ          # perímetro do círculo
razao_circ = P_circ / A              # razão perímetro/área do círculo
razao_circ

print(f"Cidade: {city}, Área: {area:.4f}, Perímetro: {per:.4f}, Razão per/area: {per/area:.4f}, Razão círculo: {razao_circ:.4f}")
# Cidade: londrina, Área: 106.9872, Perímetro: 513.9772, Razão per/area: 4.8041, Razão círculo: 0.3427
# Cidade: ibipora, Área: 14.6147, Perímetro: 102.6641, Razão per/area: 7.0247, Razão círculo: 0.9273
# Cidade: cambe, Área: 26.3545, Perímetro: 147.7575, Razão per/area: 5.6065, Razão círculo: 0.6905
# Cidade: apucarana, Área: 34.2368, Perímetro: 208.1739, Razão per/area: 6.0804, Razão círculo: 0.6058
# Cidade: arapongas, Área: 33.7098, Perímetro: 211.0196, Razão per/area: 6.2599, Razão círculo: 0.6106
# Cidade: cambira, Área: 2.0964, Perímetro: 13.7907, Razão per/area: 6.5783, Razão círculo: 2.4483
# Cidade: jandaia, Área: 6.9600, Perímetro: 41.8099, Razão per/area: 6.0072, Razão círculo: 1.3437
# Cidade: mandaguari, Área: 9.8709, Perímetro: 63.2621, Razão per/area: 6.4089, Razão círculo: 1.1283
# Cidade: marialva, Área: 9.8590, Perímetro: 70.7047, Razão per/area: 7.1716, Razão círculo: 1.1290
# Cidade: rolandia, Área: 19.2268, Perímetro: 106.8232, Razão per/area: 5.5560, Razão círculo: 0.8084
# Cidade: sarandi, Área: 19.0591, Perímetro: 63.0432, Razão per/area: 3.3078, Razão círculo: 0.8120
# Cidade: maringa, Área: 91.4141, Perímetro: 357.0259, Razão per/area: 3.9056, Razão círculo: 0.3708

#INDUSTRIAL
# Cidade: londrina, Área: 20.5565, Perímetro: 366.2197, Razão per/area: 17.8153, Razão círculo: 0.7819
# Cidade: ibipora, Área: 3.3302, Perímetro: 62.3865, Razão per/area: 18.7333, Razão círculo: 1.9425
# Cidade: cambe, Área: 5.8699, Perímetro: 105.9476, Razão per/area: 18.0494, Razão círculo: 1.4632
# Cidade: apucarana, Área: 7.6188, Perímetro: 151.6977, Razão per/area: 19.9109, Razão círculo: 1.2843
# Cidade: arapongas, Área: 9.6673, Perímetro: 165.2695, Razão per/area: 17.0957, Razão círculo: 1.1401
# Cidade: cambira, Área: 0.2516, Perímetro: 5.4725, Razão per/area: 21.7538, Razão círculo: 7.0677
# Cidade: jandaia, Área: 1.6052, Perímetro: 31.0838, Razão per/area: 19.3641, Razão círculo: 2.7979
# Cidade: mandaguari, Área: 2.7672, Perímetro: 50.1281, Razão per/area: 18.1150, Razão círculo: 2.1310
# Cidade: marialva, Área: 2.8511, Perímetro: 58.4463, Razão per/area: 20.4997, Razão círculo: 2.0994
# Cidade: rolandia, Área: 4.8037, Perímetro: 82.3064, Razão per/area: 17.1340, Razão círculo: 1.6174
# Cidade: sarandi, Área: 2.6594, Perímetro: 49.9092, Razão per/area: 18.7671, Razão círculo: 2.1738
# Cidade: maringa, Área: 23.3357, Perímetro: 390.5176, Razão per/area: 16.7348, Razão círculo: 0.7338

Cidade: maringa, Área: 23.3357, Perímetro: 390.5176, Razão per/area: 16.7348, Razão círculo: 0.7338
